In [1]:
from detect_intermediates import *
from interpret_tICs import *

In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# changing matplotlib the default style
matplotlib.style.use('ggplot')

from PDB_Order_Fixer import PDB_Order_Fixer
import mdtraj as md
import os
import numpy as np
import h5py

import datetime
import glob
import copy
from functools import partial 
import operator
import time

import random 
import subprocess
from subprocess import Popen
import sys
from io_functions import *
from custom_clusterer import *
from custom_tica import *
from custom_featurizer import *
from pdb_editing import *
from analysis import *
from io_functions import *
#from topology_fixing import *
from subsampling import *
from conversions import *
from custom_msm import *
#from grids import *
from docking_analysis import *

/home/enf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
import io_functions
from imp import reload
reload(io_functions)
from io_functions import *

In [4]:
from operator import itemgetter

In [5]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [52]:
from get_variable_names import *
inactive_dir='/home/enf/md_simulations/B1AR/reference_receptors/5f8u_renumbered_P_for_conformation.pdb'
active_dir='/home/enf/md_simulations/B1AR/reference_receptors/5f8u_hm_P_for_conformation.pdb'
simulation_structure="/home/enf/md_simulations/B1AR/reference_receptors/run0_0.pdb"
common_residues_pkl='/home/enf/md_simulations/B1AR/reference_receptors/common_residues.pkl'
contact_residues = find_common_residues([inactive_dir, active_dir, simulation_structure], common_residues_pkl)


[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 

In [8]:
import os
import sys
import csv


# all the parameters

exacycle = False

n_clusters = 300
lag_time = 5
if exacycle:
  lag_time *= 2

msm_lag_time = 5
n_components = 10
n_samples = 10
n_macrostates = 25
n_trees = 100

precision = "SP"

sparse = True
wolf = True

is_sparse = ""
if(sparse): is_sparse = "sparse-"

shrinkage_string = ""
rho_string = ""
if wolf and not sparse:
  wolf_string = "_wolf_"
  shrinkage = None
  shrinkage_string = "autoShrinkage"
  rho = None
  rho_string = ""
elif wolf and sparse:
  wolf_string = "_wolf_"
  shrinkage = None
  shrinkage_string = "autoShrinkage"
  rho = 0.005
  rho_string = "_rho0pt005"
else:
  wolf_string = ""
  shrinkage = 0.001
  shrinkage_string = "0pt001"
  rho = None
  rho_string = ""

traj_ext = ".h5"
base = "/home/enf/md_simulations/B1AR"
traj_dir = "/home/enf/md_simulations/B1AR/h5_trajectories"
structure = None
iterative = False
featurize_parallel = True
sampling_method = "random"
precision = "SP"

In this iPython notebook, we will featurize MOR ligand binding simulation by pairwise distances between the ligand and different receptor residues. We will then perform tICA and prospectively build an MSM. 

In [9]:
feature_name="contact_6pt6_angstroms"
ori_feature_name = copy.deepcopy(feature_name)

In [12]:
lag_time = 5
n_components = 2
n_clusters = 25

rho = 0.01 # tica regularization parameter
rho_string = "0pt01"
lag_time=50

In [13]:
schemes = ["closest-heavy", "CA"]
feature_name = "%s-CA-py3-far" %ori_feature_name
#feature_name = "%s_phi_psi_chi2" %feature_name
(active_ref_dir, inactive_ref_dir, simulation_ref_dir, scripts_dir,
          ligand_dir, agonist_dir, inverse_agonist_dir, biased_agonist_dir, ref_receptors_dir, whole_trajectory_pnas,
          sasa_file) = get_base_files(base)

tica_dir = get_tica_dir(base, is_sparse, lag_time, n_components, feature_name, 
                                 wolf_string, shrinkage_string, rho_string)
#tica_dir = "%s-with-inactive" %tica_dir
tica_dir = "%s-backup" %tica_dir
ori_tica_dir = copy.deepcopy(tica_dir)
#tica_dir = "%s-normalized" % ori_tica_dir
features_dir = get_features_dir(base, feature_name)

landmarks_dir = get_landmarks_dir(tica_dir)
analysis_dir = get_analysis_dir(tica_dir, n_clusters, sampling_method)
gmm_dir = get_gmm_dir(tica_dir)
rf_dir = get_rf_dir(tica_dir)


ref_tica_dir, ref_tica_coords = get_ref_tica_dirs(tica_dir)

graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)

pnas_titles =  ["tm6_tm3_dist", "rmsd_npxxy_inactive", "rmsd_npxxy_active", "rmsd_connector_inactive", "rmsd_connector_active"]
pnas_features_dir = analysis_dir


(clusterer_dir, msm_model_dir, macrostate_dir, features_known, model_dir, projected_features_dir,
         projection_operator_dir, ktica_fit_model_filename, ktica_projected_data_filename, nystroem_data_filename,
         mutual_information_csv, pearson_csv) = get_tica_files(base, tica_dir, n_clusters, msm_lag_time, n_macrostates)

(standardized_features_dir, feature_residues_csv, feature_residues_pkl,
          contact_csv, ref_features_dir) = get_feature_files(features_dir)

(kmeans_csv, tica_coords_csv, features_csv, active_rmsd_dir, inactive_rmsd_dir, active_pnas_dir, inactive_pnas_joined, active_pnas_joined,
        clusters_map_file, ktica_clusters_map_file, analysis_file, combined_file, docking_summary, docking_joined, docking_z_scores_csv,
        aggregate_docking, aggregate_docking_joined, docking_pnas_joined, aggregate_docking_pnas, aggregate_docking_pnas_joined, docking_multiple_ligands,
        docking_distances_file, docking_pdf, mmgbsa_docking_distances, pnas_coords, mmgbsa_dir, mmgbsa_csv, mmgbsa_pdf, aggregate_mmgbsa,
        aggregate_mmgbsa_joined, aggregate_mmgbsa_pnas_joined, mmgbsa_z_scores_csv, active_clusters_csv, intermediate_clusters_csv,
        inactive_clusters_csv, pnas_clusters_averages, tica_clusters_averages, tica_classes_csv, tica_samples_csv, subgraph_save_base,
        degree_save_base, degree_map_csv, degree_z_map_csv, aggregate_docking_pnas_degree_z_joined, tic_residue_csv, feature_coefs_csv,
        duplicated_feature_coefs_csv) = get_analysis_files(analysis_dir, n_clusters, tica_dir, tica_dir, sampling_method, n_samples, precision,
                                                           msm_lag_time)

(inactive_pnas_distances_dir, active_pnas_distances_dir, active_pnas_all_distances_dir,
          inactive_pnas_distances_new_csv, active_pnas_distances_new_csv, active_pnas_joined, active_pnas_means, pnas_coords_dir,
          pnas_coords_csv, pnas_all_coords_csv, pnas_coords_hexbin_dir, pnas_coords_co_crystallized_docking_dir,
          pnas_coords_active_colors_dir, user_defined_features_file, reaction_coordinates_trajs_file) = get_pnas_files(whole_trajectory_pnas, pnas_features_dir)

features_dir = get_features_dir(base, feature_name)



graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)
(scripts_dir, pymol_fixpdb_dir) = get_script_dir(scripts_dir)
(save_dir, reimaged_dir, mae_dir, combined_reimaged_dir, grid_dir, docking_dir) = get_docking_dirs(tica_dir, n_clusters, n_components, n_samples, sampling_method, precision)


/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far
/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far


In [17]:
""" Parallel - do this later
from ipyparallel import Client
rc = Client()
print(len(rc.ids))
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))
"""

Waiting for connection file: /vspdata/vspcompute/home/clarezhu/.ipython/profile_default/security/ipcontroller-client.json


OSError: Connection file '/vspdata/vspcompute/home/clarezhu/.ipython/profile_default/security/ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

In [15]:
"""
user_defined_features_dir = "%s/user_defined_features" %traj_dir
if not os.path.exists(user_defined_features_dir):
    os.makedirs(user_defined_features_dir)
compute_user_defined_features_wrapper(traj_dir, traj_ext, inactive_dir, active_dir, None,
                                        feature_name_residues_dict, user_defined_features_file, worker_pool=None, 
                                        save_dir=user_defined_features_dir, parallel=True, overwrite=False)
"""

'\nuser_defined_features_dir = "%s/user_defined_features" %traj_dir\nif not os.path.exists(user_defined_features_dir):\n    os.makedirs(user_defined_features_dir)\ncompute_user_defined_features_wrapper(traj_dir, traj_ext, inactive_dir, active_dir, None,\n                                        feature_name_residues_dict, user_defined_features_file, worker_pool=None, \n                                        save_dir=user_defined_features_dir, parallel=True, overwrite=False)\n'

In [16]:
"""
ref_user_defined_features_file = "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5"
ref_user_defined_features_dir = "%s/ref_user_defined_features" %traj_dir
if not os.path.exists(ref_user_defined_features_dir):
    os.makedirs(ref_user_defined_features_dir)

compute_user_defined_features_wrapper(ref_receptors_dir, ".pdb", inactive_dir, active_dir, None,
                                        feature_name_residues_dict, ref_user_defined_features_file, worker_pool=None, 
                                        save_dir=ref_user_defined_features_dir, parallel=False, overwrite=True)
"""

'\nref_user_defined_features_file = "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5"\nref_user_defined_features_dir = "%s/ref_user_defined_features" %traj_dir\nif not os.path.exists(ref_user_defined_features_dir):\n    os.makedirs(ref_user_defined_features_dir)\n\ncompute_user_defined_features_wrapper(ref_receptors_dir, ".pdb", inactive_dir, active_dir, None,\n                                        feature_name_residues_dict, ref_user_defined_features_file, worker_pool=None, \n                                        save_dir=ref_user_defined_features_dir, parallel=False, overwrite=True)\n'

In [37]:
# don't run this every time - takes time


import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *
cutoff = 0.66

# output
# ---> permutations of tuple pairs (res1, res2)
# |   distance(res1, res2) at time t
# |
# time t
featurize_contacts_custom(traj_dir, features_dir = features_dir, traj_ext = traj_ext, contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir, simulation_structure], contact_residues=contact_residues,
                          residues_map = None, contact_cutoff = cutoff, parallel=True, exacycle = exacycle, traj_top_structure = None, iterative=False,
                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes, binarize=None, redo=False, within_turn=False) # removed worker_pool = dview, added parallel = True


structure
/home/enf/md_simulations/B1AR/reference_receptors/5f8u_renumbered_P_for_conformation.pdb
structure
None
mdraj_index_combinations[0:10]
[(189, 135), (189, 51), (189, 30), (189, 116), (189, 210), (189, 142), (189, 265), (189, 165), (189, 115), (189, 34)]
contact_features[0:10]
[[LEU191, PHE137], [LEU191, VAL53], [LEU191, THR32], [LEU191, ARG118], [LEU191, MET212], [LEU191, TRP144], [LEU191, LYS267], [LEU191, ASN167], [LEU191, THR117], [LEU191, GLN36]]
About to compute 35515 features
(1, 35515)
cutoff
0.66
distances[0:10]
[[ 2.92911363  2.50330114  2.82146573 ...,  1.06045723  1.05432832
   1.47340202]]
There are 1033 residue-residue contacts below cutoff in structure.
structure
/home/enf/md_simulations/B1AR/reference_receptors/5f8u_hm_P_for_conformation.pdb
structure
None
mdraj_index_combinations[0:10]
[(192, 138), (192, 54), (192, 33), (192, 119), (192, 213), (192, 145), (192, 268), (192, 168), (192, 118), (192, 37)]
contact_features[0:10]
[[LEU191, PHE137], [LEU191, VAL53], [

In [40]:
# returns ordered list of every dataset file (features)
feature_list = get_trajectory_files(features_dir, ".dataset")

In [57]:
n_frames = 0

for ds in feature_list:
    f = load_file(ds)
    n_frames += f.shape[0]
    print(f.shape[1])
    break

n_frames
# featurized_trajs = load_file_list(get_trajectory_files(features_dir, ".dataset"))

loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN0.dataset
2680


347

In [17]:
# [(featurized_trajs[i].shape, get_trajectory_files(traj_dir, traj_ext)[i]) for i in range(0, len(featurized_trajs))]

In [42]:
# scaling

n = StandardScaler()
for feature in featurized_trajs:
    n.partial_fit(feature)
# normalizing features
# transforms features to zero mean and unit variance (z-scoring)
save_file = "%s/normalizer.h5" %features_dir
verbosedump(n, save_file)

NameError: name 'featurized_trajs' is not defined

In [43]:
fit_normalizer(features_dir, features=None)

loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN0.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUNFAH.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM00.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM1.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM2.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM3.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM4.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM5.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM6.dataset
loading /home/enf/md

In [44]:
import pickle
with open(feature_residues_pkl, "rb") as f:
    feature_residues = pickle.load(f)

In [45]:
dir(feature_residues[0])
dir(feature_residues[0].residue_i)
feature_residues[0].residue_i.resSeq

105

In [46]:
import custom_tica
reload(custom_tica)
from custom_tica import *
# fit the tica model, apply model to each matrix
fit_and_transform(features_directory = features_dir, model_dir = tica_dir, stride=None, lag_time = lag_time, 
                  n_components = n_components, sparse = sparse, wolf = wolf, rho = rho, shrinkage = shrinkage,
                  parallel=False, traj_ext = traj_ext, normalize=True,
                  partial_fit=False, recompute_tica = False) # deleted features=featurized_trajs

loading feature files
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN0.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN0.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUNFAH.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUNFAH.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM00.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM00.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM1.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM1.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM2.dataset
loading /home/enf/md_simulations/B1

In [22]:
#bw_trajectories = [t for t in get_trajectory_files(traj_dir, ".h5") if "RUN" in t] # fixing "chain" problem

In [23]:
# chain swapping

"""
from functools import partial
from importlib import reload
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import swap_chains_in_traj

swap_chains_in_traj_partial = partial(swap_chains_in_traj, chains=("I", "A"))
dview.map_sync(swap_chains_in_traj_partial, bw_trajectories)
"""

'\nfrom functools import partial\nfrom importlib import reload\nimport custom_featurizer\nreload(custom_featurizer)\nfrom custom_featurizer import swap_chains_in_traj\n\nswap_chains_in_traj_partial = partial(swap_chains_in_traj, chains=("I", "A"))\ndview.map_sync(swap_chains_in_traj_partial, bw_trajectories)\n'

In [24]:
# [(c.id, len([r for r in c.residues if r.is_protein])) for c in t.topology.chains]

In [53]:
simulation_structure

'/home/enf/md_simulations/B1AR/reference_receptors/run0_0.pdb'

In [56]:
# featurize reference receptors (controls)

import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *

ref_receptors_dir = '/home/enf/md_simulations/B1AR/reference_receptors'
featurize_contacts_custom(ref_receptors_dir, features_dir = ref_features_dir, traj_ext = ".pdb", contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir, simulation_structure], contact_residues=contact_residues,
                          residues_map = None, contact_cutoff = cutoff, parallel=False, exacycle = exacycle, traj_top_structure = None, iterative=False, simulation_reference_filename=get_trajectory_files(traj_dir, traj_ext)[0],
                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes, binarize=None, redo=False, within_turn=False)

#featurize_contacts_custom(ref_receptors_dir, features_dir = ref_features_dir, traj_ext = ".pdb", contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
#                          residues_map = None, contact_cutoff = cutoff, parallel = False, exacycle = exacycle, traj_top_structure = None, iterative=False,
#                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes, within_turn=False, binarize=None, redo=False)

structure
/home/enf/md_simulations/B1AR/reference_receptors/5f8u_renumbered_P_for_conformation.pdb
structure
None
mdraj_index_combinations[0:10]
[(189, 135), (189, 51), (189, 30), (189, 116), (189, 210), (189, 142), (189, 265), (189, 165), (189, 115), (189, 34)]
contact_features[0:10]
[[LEU191, PHE137], [LEU191, VAL53], [LEU191, THR32], [LEU191, ARG118], [LEU191, MET212], [LEU191, TRP144], [LEU191, LYS267], [LEU191, ASN167], [LEU191, THR117], [LEU191, GLN36]]
About to compute 35515 features
(1, 35515)
cutoff
0.66
distances[0:10]
[[ 2.92911363  2.50330114  2.82146573 ...,  1.06045723  1.05432832
   1.47340202]]
There are 1033 residue-residue contacts below cutoff in structure.
structure
/home/enf/md_simulations/B1AR/reference_receptors/5f8u_hm_P_for_conformation.pdb
structure
None
mdraj_index_combinations[0:10]
[(192, 138), (192, 54), (192, 33), (192, 119), (192, 213), (192, 145), (192, 268), (192, 168), (192, 118), (192, 37)]
contact_features[0:10]
[[LEU191, PHE137], [LEU191, VAL53], [

In [48]:
not_a_thing = md.load_frame(get_trajectory_files(traj_dir, traj_ext)[0], index=0)

In [58]:
ref_features = load_file_list(get_trajectory_files(ref_features_dir, '.dataset'))
ref_features
for x in ref_features :
    print(x.shape)

None
None
['/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_hm_P_for_conformation.dataset', '/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_renumbered_P_for_conformation.dataset', '/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/run0_0.dataset']
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_hm_P_for_conformation.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/run0_0.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_renumbered_P_for_conformation.dataset
(1, 2680)
(1, 2680)
(1, 2680)


In [59]:
transform(existing_model = projection_operator_dir, features_directory = ref_features_dir, tica_dir = ref_tica_dir, normalizer=compat_verboseload("%s/normalizer.h5" %features_dir))


None
None
['/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_hm_P_for_conformation.dataset', '/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_renumbered_P_for_conformation.dataset', '/home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/run0_0.dataset']
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_hm_P_for_conformation.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/run0_0.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_renumbered_P_for_conformation.dataset


In [60]:
feature_residues

[ALA105 to GLN113,
 ALA105 to ILE100,
 ALA105 to PHE110,
 ALA105 to TYR112,
 ALA119 to ILE124,
 ALA119 to LEU38,
 ALA119 to LEU41,
 ALA12 to ALA62,
 ALA12 to LEU17,
 ALA12 to MET7,
 ALA12 to TRP246,
 ALA12 to VAL58,
 ALA121 to CYS126,
 ALA121 to LEU38,
 ALA121 to LEU41,
 ALA121 to MET116,
 ALA121 to PHE42,
 ALA121 to THR39,
 ALA121 to VAL97,
 ALA130 to ILE125,
 ALA130 to LEU138,
 ALA130 to LEU86,
 ALA130 to VAL135,
 ALA133 to GLU93,
 ALA133 to LEU138,
 ALA133 to LEU86,
 ALA133 to MET141,
 ALA133 to PRO139,
 ALA133 to SER82,
 ALA133 to THR127,
 ALA133 to THR89,
 ALA133 to VAL128,
 ALA152 to ARG146,
 ALA152 to ARG168,
 ALA152 to ARG233,
 ALA152 to ASP147,
 ALA152 to ASP158,
 ALA152 to CYS161,
 ALA152 to GLN157,
 ALA152 to THR166,
 ALA152 to VAL165,
 ALA169 to ASP147,
 ALA169 to SER174,
 ALA169 to TRP144,
 ALA169 to VAL230,
 ALA171 to ASN226,
 ALA171 to ILE176,
 ALA171 to ILE227,
 ALA171 to PHE164,
 ALA171 to PHE223,
 ALA171 to THR166,
 ALA171 to TRP144,
 ALA171 to TRP145,
 ALA171 to TYR1

In [29]:
#transform_to_h5(projection_operator_dir, features_dir, projected_features_dir, worker_pool=None, parallel=True, normalizer=compat_verboseload("%s/normalizer.h5" %features_dir))

In [61]:
tica_object = compat_verboseload(projection_operator_dir)

In [62]:
with open(projection_operator_dir, 'rb') as f:
    tica_object = pickle.load(f, encoding='latin1') 

In [63]:
#all_tica = compat_verboseload(projected_features_dir)
#all_tica = [x[:,range(1,10)] for x in all_tica]
#print(all_tica[0].shape)
#print(len(all_tica))
#verbosedump(all_tica, projected_features_dir)


In [64]:
#n_components=9

In [65]:
tica_object.timescales_

array([ 1281.90987111,   912.28999461])

In [66]:
plot_columns(tica_dir, projected_features_dir, titles = ["tIC%d" %j for j in range(1,11)], tICA = True, scale = 1.0, refcoords_file = None)

loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/phi_psi_chi2_allprot_projected.h5"...
()
None
Done plotting columns


In [67]:
feature_indices = []
traj_basenames = get_trajectory_files(traj_dir, ".h5")
traj_basenames = [os.path.splitext(os.path.basename(t))[0] for t in traj_basenames]
feature_basenames = get_trajectory_files(features_dir, ".dataset")
feature_basenames = [os.path.splitext(os.path.basename(t))[0] for t in feature_basenames]
for i, traj_basename in enumerate(traj_basenames):
    if traj_basename in feature_basenames:
        feature_indices.append(int(i))

In [68]:
tica_coords = compat_verboseload(projected_features_dir)
#pnas_coords = compat_verboseload(user_defined_features_file)
#pnas_coords = [pnas_coords[i] for i in feature_indices]
tica_names = ["tIC.%d" %i for i in range(1,n_components+1)]
#pnas_names = sorted(feature_name_residues_dict.keys())

In [69]:
# prints total number of frames that we have (t=0 - t=n)
print(np.concatenate(tica_coords).shape)
#print(np.concatenate(pnas_coords).shape)

(29570, 2)


In [70]:
from plots import *

In [71]:
#plot_data_vs_data(np.concatenate(tica_coords), np.concatenate(pnas_coords), tica_names, pnas_names, analysis_dir)

In [72]:
subsampled_features_dir = os.path.join(tica_dir, "subsampled_features")
if not os.path.exists(subsampled_features_dir): os.makedirs(subsampled_features_dir)

In [73]:
feature_names = compat_verboseload(feature_residues_pkl)

In [74]:
ref_features = [load_file(f) for f in get_trajectory_files(ref_features_dir, ".dataset")]
print(len(ref_features))
print(ref_features[0].shape)
print(ref_features[1].shape)
print(len(feature_names))
ref_features = pd.DataFrame(np.vstack(ref_features), columns=[str(f) for f in feature_names])

loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_hm_P_for_conformation.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_renumbered_P_for_conformation.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/run0_0.dataset
3
(1, 2680)
(1, 2680)
2680


In [75]:
ref_features

,ALA105 to GLN113,ALA105 to ILE100,ALA105 to PHE110,ALA105 to TYR112,ALA119 to ILE124,ALA119 to LEU38,ALA119 to LEU41,ALA12 to ALA62,ALA12 to LEU17,ALA12 to MET7,...,VAL225_CA to VAL236_CA,VAL228_CA to VAL236_CA,VAL230_CA to VAL236_CA,VAL242_CA to VAL65_CA,VAL52_CA to VAL57_CA,VAL52_CA to VAL58_CA,VAL53_CA to VAL58_CA,VAL53_CA to VAL88_CA,VAL57_CA to VAL85_CA,VAL57_CA to VAL88_CA
0,0.651959,0.488605,0.558262,0.338301,0.495288,0.661648,0.723374,0.613999,0.480660,0.478179,...,0.804015,0.763085,0.988030,0.933461,0.696834,0.859262,0.581535,0.795517,0.917281,0.982492
1,0.604994,0.500234,0.556815,0.341518,0.531865,0.565963,0.532016,0.600338,0.527701,0.492347,...,0.698533,0.766658,1.004664,0.885828,0.671757,0.870483,0.581221,0.876324,0.939267,1.060570
2,0.517473,0.490634,0.601032,0.380320,0.523140,0.663725,0.958512,0.629974,0.505724,0.508850,...,0.630061,0.788933,1.041141,1.090160,0.679401,0.878568,0.576776,0.793440,0.960139,0.981657


In [76]:
#import interpret_tICs
#reload(interpret_tICs)
#from interpret_tICs import *

tic_components_dir = tica_dir
important_contact_features = interpret_tIC_components(projection_operator_dir, tic_components_dir, feature_residues_pkl, n_tica_components=n_components, percentile=95)


Interpreting tIC 1
feature_importances_df.shape
(14, 5)
residue_importances_df.shape
(19, 2)
              feature_name      res_i      res_j  importance  \
11  Thr166_Ca To Tyr156_Ca  Thr166_Ca  Tyr156_Ca    0.498721   
4   Asn167_Ca To Tyr156_Ca  Asn167_Ca  Tyr156_Ca   -0.487269   
2   Ala171_Ca To Thr166_Ca  Ala171_Ca  Thr166_Ca    0.334199   
9   Ser174_Ca To Thr166_Ca  Ser174_Ca  Thr166_Ca    0.248095   
6   Ile172_Ca To Thr166_Ca  Ile172_Ca  Thr166_Ca   -0.214432   
1   Ala169_Ca To Asp147_Ca  Ala169_Ca  Asp147_Ca    0.207239   
12    Trp239_Ca To Trp3_Ca  Trp239_Ca    Trp3_Ca    0.140266   
3   Asn167_Ca To Ser174_Ca  Asn167_Ca  Ser174_Ca   -0.138184   
0          Leu35 To Phe265      Leu35     Phe265    0.130284   
10   Thr166_Ca To Thr81_Ca  Thr166_Ca   Thr81_Ca    0.117084   
13  Tyr170_Ca To Val165_Ca  Tyr170_Ca  Val165_Ca   -0.109186   
8   Phe241_Ca To Thr166_Ca  Phe241_Ca  Thr166_Ca    0.105903   
7   Phe215_Ca To Tyr259_Ca  Phe215_Ca  Tyr259_Ca    0.099288   
5   Asn226_

In [77]:
import interpret_tICs
reload(interpret_tICs)
from interpret_tICs import *
tic_subsampled_features_file = "%s/features_subsampled.pkl" % tica_dir
subsampled_features_dir = os.path.join(tica_dir, "subsampled_features")
if not os.path.exists(subsampled_features_dir): os.makedirs(subsampled_features_dir)
important_contact_features_pruned, important_contact_features_indices = find_non_zero_features(important_contact_features[0], feature_names)
if not os.path.exists(tic_subsampled_features_file):
    subsample_features(features_dir, important_contact_features_indices, important_contact_features_pruned, tic_subsampled_features_file)

loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN0.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM4.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM7.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_R1.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_R12.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_R15.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_Active_GP_LigandXAM1.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_R7.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_R4.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/RUN_RR15.da

In [78]:
tic_subsampled_features_file

'/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/features_subsampled.pkl'

In [79]:
protein_top_features = load_file(tic_subsampled_features_file)
protein_top_features_concat = pd.concat(protein_top_features)

loading /home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/features_subsampled.pkl


In [83]:
trajs = range(0, len(get_trajectory_files(traj_dir, ".h5")))
#trajs = [t for t in get_trajectory_files(traj_dir, traj_ext) if "apo_reimaged" not in t and "salm_nb" not in t]

#user_defined_coords = load_file(user_defined_features_file)

In [84]:
#salm_pnas = [user_defined_coords[i] for i in salm_trajs]
#apo_pnas = [user_defined_coords[i] for i in apo_trajs]
#pnas = [user_defined_coords[i] for i in trajs]
#pnas = [user_defined_coords[i] for i in trajs]
#f338a_pnas = [user_defined_coords[i] for i in f338a_trajs]

In [85]:
#salm_pnas_file = "%s/salm.h5" % whole_trajectory_pnas
#apo_pnas_file = "%s/apo.h5" % whole_trajectory_pnas
#pnas_file = "%s/salm_nb.h5" % whole_trajectory_pnas
#pnas_file = "%s/salm_nb.h5" % whole_trajectory_pnas
#f338a_pnas_file = "%s/f338a.h5" % whole_trajectory_pnas

In [86]:
"""
from imp import reload
import analysis
reload(analysis)
from analysis import *
import seaborn as sns
salm_pnas_concatenated = np.concatenate(salm_pnas)
print(np.shape(salm_pnas_concatenated))
apo_pnas_concatenated = np.concatenate(apo_pnas)
#pnas_concatenated = np.concatenate(pnas)
pnas_concatenated = np.concatenate(pnas)
f338a_pnas_concatenated = np.concatenate(f338a_pnas)

verbosedump(salm_pnas, salm_pnas_file)
verbosedump(apo_pnas, apo_pnas_file)
#verbosedump(pnas, pnas_file)
verbosedump(pnas, pnas_file)
verbosedump(f338a_pnas, f338a_pnas_file)


columns=sorted(feature_name_residues_dict.keys())
#plot_columns(whole_trajectory_pnas, salm_pnas_file, titles=columns, main = "salm", tICA = False, scale = 1.0, refcoords_file = None)
#plot_columns(whole_trajectory_pnas, apo_pnas_file, titles=columns, main = "Apo", tICA = False, scale = 1.0, refcoords_file = None)

#df = pd.DataFrame(salm_pnas_concatenated, columns=sorted(feature_name_residues_dict.keys()))
#sns.jointplot(x="tm6_tm3_dist", y="rmsd_npxxy_inactive", data=df, kind="kde")
#plt.hexbin(salm_pnas_concatenated[:,0], salm_pnas_concatenated[:,2], cmap=plt.cm.YlOrRd_r)
"""

'\nfrom imp import reload\nimport analysis\nreload(analysis)\nfrom analysis import *\nimport seaborn as sns\nsalm_pnas_concatenated = np.concatenate(salm_pnas)\nprint(np.shape(salm_pnas_concatenated))\napo_pnas_concatenated = np.concatenate(apo_pnas)\n#pnas_concatenated = np.concatenate(pnas)\npnas_concatenated = np.concatenate(pnas)\nf338a_pnas_concatenated = np.concatenate(f338a_pnas)\n\nverbosedump(salm_pnas, salm_pnas_file)\nverbosedump(apo_pnas, apo_pnas_file)\n#verbosedump(pnas, pnas_file)\nverbosedump(pnas, pnas_file)\nverbosedump(f338a_pnas, f338a_pnas_file)\n\n\ncolumns=sorted(feature_name_residues_dict.keys())\n#plot_columns(whole_trajectory_pnas, salm_pnas_file, titles=columns, main = "salm", tICA = False, scale = 1.0, refcoords_file = None)\n#plot_columns(whole_trajectory_pnas, apo_pnas_file, titles=columns, main = "Apo", tICA = False, scale = 1.0, refcoords_file = None)\n\n#df = pd.DataFrame(salm_pnas_concatenated, columns=sorted(feature_name_residues_dict.keys()))\n#sns.j

In [87]:
tica_file = "%s/tica.h5" % tica_dir

In [88]:
tica_coords = compat_verboseload(projected_features_dir)
tica = [tica_coords[i] for i in trajs]
top_features = [protein_top_features[i] for i in trajs]

verbosedump(tica, tica_file)

columns = ["tIC%d" %i for i in range(1,n_components+1)]
#plot_columns(tica_dir, salm_tica_file, titles=columns, main = "salm", tICA = False, scale = 1.0, refcoords_file = None)
#plot_columns(tica_dir, apo_tica_file, titles=columns, main = "Apo", tICA = False, scale = 1.0, refcoords_file = None)


Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/tica.h5"... (<class 'list'>)


In [89]:
print(np.concatenate(tica).shape)

(29570, 2)


In [91]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *
n_clusters = 25
clusterer_dir = "%s/clusterer_%dclusters.h5" %(tica_dir, n_clusters)
if not os.path.exists(clusterer_dir):
    cluster_minikmeans(tica_dir, projected_features_dir, traj_dir, n_clusters=n_clusters, clusterer_dir=clusterer_dir)

Clustering by KMeans
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/phi_psi_chi2_allprot_projected.h5"...
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters.h5"... (<class 'msmbuilder.cluster.MiniBatchKMeans'>)


In [92]:
print(clusterer_dir)

/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters.h5


In [93]:
clusterer_file = "%s/clusterer%d.h5" %(tica_dir, n_clusters)

In [94]:
clusterer = compat_verboseload(clusterer_dir)

In [96]:
import custom_msm
reload(custom_msm)
from custom_msm import *
from msm_resampled import *

In [97]:
clusters_map = make_clusters_map(clusterer)

2706
203
1368
439
965
190
865
1858
1643
177
136
744
1429
264
1316
1139
1231
1594
1453
1685
2492
2115
1003
209
2346


In [91]:
#plt.clf()
#plot_timescales(apo_clusterer_file, n_clusters, tica_dir, main="Apo MSM Timescales", lag_times=list(range(1,51,10)))

In [92]:
#plt.clf()
#import custom_msm
#reload(custom_msm)
#from custom_msm import *
#plot_timescales(salm_clusterer_file, n_clusters, tica_dir, main="salm MSM Timescales", lag_times=[1,5,10,15,20])

In [93]:
#plt.clf()
#plot_timescales(clusterer_file, n_clusters, tica_dir, main="salm_nb MSM Timescales", lag_times=list(range(1,51,10)))

In [104]:
lag_time=25
prior_counts = 1e-5
ergodic_cutoff='off'
msm_dir = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d.h5" %(lag_time, str(prior_counts), n_clusters))
msm_object = build_msm(clusterer_dir, lag_time, msm_dir, prior_counts, ergodic_cutoff)

import msm_resampled
reload(msm_resampled)
from msm_resampled import *
total_samples = 20000
num_trajs = len(trajs)
resampled_traj_to_frames_file = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
#if not os.path.exists(resampled_traj_to_frames_file):
resampled_traj_to_frames = resample_by_msm(total_samples, msm_object, clusters_map, num_trajs, resampled_traj_to_frames_file)
#else:
resampled_traj_to_frames = compat_verboseload(resampled_traj_to_frames_file)

#pnas = compat_verboseload(pnas_file)
#pnas_resampled_file = os.path.join(tica_dir, "pnas_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
#resample_features_by_msm_equilibirum_pop(pnas, resampled_traj_to_frames, pnas_resampled_file)
#pnas_resampled = compat_verboseload(pnas_resampled_file)

tica_concatenated = np.concatenate(compat_verboseload(projected_features_dir))
tica_axes = [(np.min(tica_concatenated[:,i]), np.max(tica_concatenated[:,i])) for i in range(0, np.shape(tica_concatenated)[1])]

tica = compat_verboseload(tica_file)
tica_resampled_file = os.path.join(tica_dir, "tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(tica, resampled_traj_to_frames, tica_resampled_file)
tica_resampled = compat_verboseload(tica_resampled_file)

msm = compat_verboseload(msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = msm.populations_

for cluster_id in msm.mapping_.keys():
    state_id = msm.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)
    
#plot_all_tics_and_clusters(tica_dir, tica_resampled_file, clusterer_file, lag_time, tic_range=range(0,n_components), main = "MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)

#plot_columns(tica_dir, tica_resampled_file, titles = ["tIC.%d" %i for i in range(1,n_components+1)], main = "mOR_MSM_", tICA = False, scale = 1.0, refcoords_file = ref_tica_coords, axes=tica_axes, concatenate=False, reshape=False)

loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters.h5"...
fitting msm to trajectories with 25 clusters and lag_time 25
MarkovStateModel(ergodic_cutoff='off', lag_time=25, n_timescales=None,
         prior_counts=1e-05, reversible_type='mle', sliding_window=True,
         verbose=True)
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/msm_lag-time25_prior-counts1e-05_clusters25.h5"... (<class 'msmbuilder.msm.msm.MarkovStateModel'>)
fitted msm to trajectories with 25 states
Found number to sample per cluster based on equilibrium proporrtions.
Obtained random (trajectory, frame) pairs based on equilibrium populations
Rearranged equilibrium sampled frames based on trajectories
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_re

In [105]:
clusterer_dir

'/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters.h5'

In [107]:
tica_resampled = np.vstack([compat_verboseload(tica_resampled_file)[::1,]])
mins = np.min(tica_resampled, axis=0)
maxs = np.max(tica_resampled, axis=0)
stds = np.std(tica_resampled, axis=0)
custom_lims = [[mins[i] - 2*stds[i], maxs[i] + 2*stds[i]] for i in range(0,len(mins))]

In [108]:
#ref_pnas = load_file("/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5")
#ref_pnas = np.hstack([np.array(x).T for x in ref_pnas])

In [109]:
def get_features(features_dir):
    feature_files = get_trajectory_files(features_dir, ".dataset")
    features = np.vstack([load_file(f) for f in feature_files])
    print(features.shape)
    return features

ref_pp_features = pd.DataFrame(get_features(ref_features_dir), columns=feature_names)

ref_pp_tica = pd.read_csv(ref_tica_coords, header=None).values
#ref_pp_tica.columns = ["mOR tIC.%d" %i for i in range(1,ref_pp_tica.shape[1])]

#ref_pnas = load_file("/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5")
#ref_pnas = np.hstack([np.array(x).T for x in ref_pnas])

#ref_pnas = pd.DataFrame(ref_pnas, columns=sorted(feature_name_residues_dict.keys()))

#ref_features = pd.concat([ref_pp_features*10., ref_lp_features*10., ref_pp_tica, ref_lp_tica, ref_pnas], axis=1)

loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_hm_P_for_conformation.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/5f8u_renumbered_P_for_conformation.dataset
loading /home/enf/md_simulations/B1AR/featurescontact_6pt6_angstroms-CA-py3-far/reference_receptors/run0_0.dataset
(3, 2680)


In [110]:
#coords_bounds_dict = {"tIC.4": [(8.0, 10.), (12., 15.)]}
#all_traj_files = get_trajectory_files(traj_dir, traj_ext)
#trajs_to_watch = reaction_coordinate_sampler([all_traj_files[i] for i in salm_trajs], ".h5", [t.values for t in all_traj_features], 
#                            all_traj_features[0].columns.values.tolist(), coords_bounds_dict, 
#                            "%s/trajs_to_watch.csv" %pp_tica_dir)

In [114]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *
jointplots(tica_resampled[::1,:], 
           analysis_dir, titles = ["tIC %d" %i for i in range(1, n_components+1)], 
           main = "MSM_%d_Clusters" %n_clusters, refcoords = ref_pp_tica, 
           axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=10, 
           min_density=None, max_diff=10., max_i = 1, custom_lims=custom_lims, n_levels=11, 
           n_pts=200j, parallel=True, remake=True)

Making delta G plots.
hi
(2, 19998)
Computed 2D KDE
min_density=0.0
Computed Density. Now plotting.
(0.0, 10.0)
Done plotting columns


In [112]:
plt.show()

In [ ]:
salm_clusterer = compat_verboseload(salm_clusterer_file)
salm_msm = compat_verboseload(salm_msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = salm_msm.populations_

for cluster_id in salm_msm.mapping_.keys():
    state_id = salm_msm.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)

#plot_all_tics_and_clusters(tica_dir, salm_tica_resampled_file, salm_clusterer_file, lag_time, tic_range=[0,3], main = "salm_MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)


In [ ]:
len(dview)

In [109]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 1
n_clusters= 1000
samples_dir = "%s/salm_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/salm_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(salm_clusterer_file, salm_tica_file, [traj_files[i] for i in salm_trajs], 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=None, parallel=True, lig_name="LIG")

/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/salm_clusterer_1000clusters_1samples_samples_kdtree
loading "/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/salm_clusterer1000.h5"...
loading "/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/salm_tica.h5"...
loading "/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/salm_clusterer1000.h5"...
[[ 42  53]
 [177  73]
 [ 79 144]
 ..., 
 [ 92  84]
 [174 544]
 [155  23]]
1000
()
Converting to list
()
[42, 53, 0, 0]
42
Converting to list
[110, 147, 21, 0]
()
110
Converting to list
[171, 349, 42, 0]
()
171
Converting to list
[162, 32, 63, 0]
()

In [113]:
#do this + with apo
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 1
n_clusters= 25
samples_dir = "%s/clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(clusterer_dir, tica_file, [traj_files[i] for i in trajs], 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=None, parallel=True, lig_name="LIG")

/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters_1samples_samples_kdtree
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters.h5"...
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/tica.h5"...
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters.h5"...
[[104 400]
 [ 20 552]
 [ 64  18]
 [ 16 604]
 [ 68  39]
 [ 20 785]
 [  5  54]
 [ 63  44]
 [ 24 601]
 [ 20 170]
 [ 16 159]
 [ 24 127]
 [ 98 206]
 [ 20 389]
 [103 178]
 [ 15 130]
 [ 44 133]
 [101 438]
 [ 26  17]
 [ 86  55]
 [ 58  67]
 [ 37  24]
 [ 78  43]
 [ 23 334]
 [ 82  94]]
25
()
()
()
()
()
()
Convert

In [108]:
analysis_dir

'/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/analysis_n_clusters100_random'

In [ ]:
get_trajectory_files(traj_dir, traj_ext)

In [111]:
#do this + with apo
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 1
n_clusters= 1000
samples_dir = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(apo_clusterer_file, apo_tica_file, [traj_files[i] for i in apo_trajs], 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=None, parallel=True, lig_name="LIG")

/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/apo_clusterer_1000clusters_1samples_samples_kdtree
loading "/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/apo_clusterer1000.h5"...
loading "/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/apo_tica.h5"...
loading "/home/enf/md_simulations/b2ar/Salmeterol/sparse-tICA_t50_n_components10contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/apo_clusterer1000.h5"...
[[108  44]
 [127   1]
 [ 16  70]
 ..., 
 [155  81]
 [170 962]
 [ 48  51]]
1000
()
()
()
()
()
()
()
()
Converting to list
Converting to list
Converting to list
Converting to list
Converting to list
()
()
()
Converting to list
Converting to list
()
[108

In [115]:
pp_n_components = 10

salm_tica_dfs = []
for array in salm_tica:
    salm_tica_dfs.append(pd.DataFrame(array, columns=["tIC.%d" %i for i in range(1,pp_n_components+1)]))
tica_dfs = []
for array in tica:
    tica_dfs.append(pd.DataFrame(array, columns=["tIC.%d" %i for i in range(1,n_components+1)]))
apo_tica_dfs = []
for array in apo_tica:
    apo_tica_dfs.append(pd.DataFrame(array, columns=["tIC.%d" %i for i in range(1,pp_n_components+1)]))
    
    
salm_traj_features = [pd.concat([salm_tica_dfs[i], salm_top_features[i]], axis=1) for i in range(0, len(salm_tica_dfs))]
traj_features = [pd.concat([tica_dfs[i], top_features[i]], axis=1) for i in range(0, len(tica_dfs))]
apo_traj_features = [pd.concat([apo_tica_dfs[i], apo_top_features[i]], axis=1) for i in range(0, len(apo_tica_dfs))]

salm_traj_features_np = [f.values for f in salm_traj_features]
salm_features_averages = calculate_cluster_averages_per_feature(salm_clusterer, salm_traj_features_np)
salm_features_averages = pd.DataFrame(salm_features_averages, columns=salm_traj_features[0].columns)

traj_features_np = [f.values for f in traj_features]
features_averages = calculate_cluster_averages_per_feature(clusterer, traj_features_np)
features_averages = pd.DataFrame(features_averages, columns=traj_features[0].columns)

apo_traj_features_np = [f.values for f in apo_traj_features]
apo_features_averages = calculate_cluster_averages_per_feature(apo_clusterer, apo_traj_features_np)
apo_features_averages = pd.DataFrame(apo_features_averages, columns=apo_traj_features[0].columns)

In [117]:
import detect_intermediates
reload(detect_intermediates)
from detect_intermediates import *

intermediates = apo_features_averages.loc[(apo_features_averages["tIC.1"] > 6) & (apo_features_averages["tIC.6"] > 0)].index.values

intermediates = intermediates.tolist()
print(intermediates)
counts = [len(salm_clusters_map[i]) for i in range(0, n_clusters)]
for c in np.argsort(-1.0*np.array(counts)):
    if c in intermediates:
        print(c)

[6, 266, 580, 899]
6
899
580
266
